In [148]:
import numpy as np
import tensorflow as tf
import pandas as pd
from tensorflow import keras
import matplotlib.pyplot as plt

% matplotlib inline

In [149]:
tf.enable_eager_execution()

In [150]:
dataset = pd.read_csv('./dataset/HAM10000_metadata.csv')

In [151]:
print(set(dataset['dx']))
print(set(dataset['dx_type']))

{'vasc', 'df', 'bcc', 'bkl', 'akiec', 'mel', 'nv'}
{'confocal', 'consensus', 'follow_up', 'histo'}


In [152]:
dataset.head()

,lesion_id,image_id,dx,dx_type,age,sex,localization
0,HAM_0000118,ISIC_0027419,bkl,histo,80.0,male,scalp
1,HAM_0000118,ISIC_0025030,bkl,histo,80.0,male,scalp
2,HAM_0002730,ISIC_0026769,bkl,histo,80.0,male,scalp
3,HAM_0002730,ISIC_0025661,bkl,histo,80.0,male,scalp
4,HAM_0001466,ISIC_0031633,bkl,histo,75.0,male,ear


In [153]:
import pathlib

In [154]:
data_root = pathlib.Path('./dataset/')

In [155]:
for item in data_root.iterdir():
    print(item)

dataset/.DS_Store
dataset/MANIFEST.TXT
dataset/HAM10000
dataset/.ipynb_checkpoints
dataset/HAM10000_metadata.csv


In [156]:
all_image_path =list(data_root.glob('*/*'))
all_image_path = [str(path) for path in all_image_path]

img_count = len(all_image_path)
img_count

10016

In [157]:
path_ds = tf.data.Dataset.from_tensor_slices(all_image_path)

In [158]:
all_image_path[4990:5020]

['dataset/HAM10000/ISIC_0026085.jpg',
 'dataset/HAM10000/ISIC_0024692.jpg',
 'dataset/HAM10000/ISIC_0033488.jpg',
 'dataset/HAM10000/ISIC_0032796.jpg',
 'dataset/HAM10000/ISIC_0028916.jpg',
 'dataset/HAM10000/ISIC_0030181.jpg',
 'dataset/HAM10000/ISIC_0032966.jpg',
 'dataset/HAM10000/ISIC_0026913.jpg',
 'dataset/HAM10000/ISIC_0028080.jpg',
 'dataset/HAM10000/ISIC_0030817.jpg',
 'dataset/HAM10000/ISIC_0024862.jpg',
 'dataset/HAM10000/ISIC_0024876.jpg',
 'dataset/HAM10000/ISIC_0028094.jpg',
 'dataset/HAM10000/ISIC_0030803.jpg',
 'dataset/HAM10000/ISIC_0026907.jpg',
 'dataset/HAM10000/ISIC_0032972.jpg',
 'dataset/HAM10000/ISIC_0028933.jpg',
 'dataset/HAM10000/ISIC_0029393.jpg',
 'dataset/HAM10000/ISIC_0029387.jpg',
 'dataset/HAM10000/ISIC_0028099.jpg',
 'dataset/HAM10000/ISIC_0028927.jpg',
 'dataset/HAM10000/ISIC_0026922.jpg',
 'dataset/HAM10000/ISIC_0030198.jpg',
 'dataset/HAM10000/ISIC_0031286.jpg',
 'dataset/HAM10000/ISIC_0033491.jpg',
 'dataset/HAM10000/ISIC_0032957.jpg',
 'dataset/HA

In [159]:
import IPython.display as display
import random

In [160]:
#for n in range(1):
 #   image_path = random.choice(all_image_path)
  #  display.display(display.Image(image_path))

In [161]:
dataset['image_path'] = ['dataset/HAM10000/' + path +'.jpg' for path in dataset['image_id']]
dataset.head()

,lesion_id,image_id,dx,dx_type,age,sex,localization,image_path
0,HAM_0000118,ISIC_0027419,bkl,histo,80.0,male,scalp,dataset/HAM10000/ISIC_0027419.jpg
1,HAM_0000118,ISIC_0025030,bkl,histo,80.0,male,scalp,dataset/HAM10000/ISIC_0025030.jpg
2,HAM_0002730,ISIC_0026769,bkl,histo,80.0,male,scalp,dataset/HAM10000/ISIC_0026769.jpg
3,HAM_0002730,ISIC_0025661,bkl,histo,80.0,male,scalp,dataset/HAM10000/ISIC_0025661.jpg
4,HAM_0001466,ISIC_0031633,bkl,histo,75.0,male,ear,dataset/HAM10000/ISIC_0031633.jpg


In [162]:
img_path = dataset['image_path'][4]
img_path

'dataset/HAM10000/ISIC_0031633.jpg'

In [163]:
img_raw = tf.read_file(img_path)

In [164]:
img_tensor = tf.image.decode_image(img_raw)

print(img_tensor.shape)
print(img_tensor.dtype)

(450, 600, 3)
<dtype: 'uint8'>


In [165]:
image_generator = keras.preprocessing.image.ImageDataGenerator(rescale=1/255)

In [166]:
image_data = image_generator.flow_from_directory(str(data_root))

Found 10015 images belonging to 2 classes.


In [167]:
for image_batch,label_batch in image_data:
    print("Image batch shape: ", image_batch.shape)
    print("Labe batch shape: ", label_batch.shape)
    break

Image batch shape:  (32, 256, 256, 3)
Labe batch shape:  (32, 2)


# Data Processing

In [168]:
def preprocess_image(image):
    image = tf.image.decode_jpeg(image,channels = 3)
    image = tf.image.resize_images(image,[225, 300])
    image /= 255.0
    return image

In [169]:
def load_and_preprocess_image(path):
    image = tf.read_file(path)
    return preprocess_image(image)

In [170]:
all_image_paths = dataset['image_path']
all_image_labels = dataset['dx']

In [171]:
path_ds = tf.data.Dataset.from_tensor_slices(all_image_paths)

In [172]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

In [173]:
image_dataset = path_ds.map(load_and_preprocess_image,num_parallel_calls=AUTOTUNE)

In [174]:
from sklearn.preprocessing import LabelEncoder

In [175]:
label_encoder = LabelEncoder()
encodedlabel = label_encoder.fit_transform(all_image_labels)

In [176]:
all_image_labels = encodedlabel

In [177]:
set(encodedlabel)

{0, 1, 2, 3, 4, 5, 6}

In [178]:
label_ds = tf.data.Dataset.from_tensor_slices(tf.cast(encodedlabel, tf.int64))

In [179]:
image_label_ds = tf.data.Dataset.zip((image_dataset,label_ds))

In [180]:
print('image shape: ', image_label_ds.output_shapes[0])

image shape:  (225, 300, 3)


In [181]:

ds = tf.data.Dataset.from_tensor_slices((all_image_paths,all_image_labels))

def load_and_preprocess_from_path_label(path,label):
    return load_and_preprocess_image(path), label

image_label_ds = ds.map(load_and_preprocess_from_path_label)
image_label_ds

<DatasetV1Adapter shapes: ((225, 300, 3), ()), types: (tf.float32, tf.int64)>

# Training

In [182]:

from tensorflow.keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D
from tensorflow.keras.initializers import glorot_uniform
from tensorflow.keras.models import Model


In [183]:
BATCH_SIZE = 32
image_count = len(dataset)
ds = image_label_ds.shuffle(buffer_size=image_count)
ds = ds.repeat()
ds = ds.batch(BATCH_SIZE)

ds = ds.prefetch(buffer_size=AUTOTUNE)
ds

<DatasetV1Adapter shapes: ((?, 225, 300, 3), (?,)), types: (tf.float32, tf.int64)>

In [184]:
# GRADED FUNCTION: identity_block

def identity_block(X, f, filters, stage, block):
    """
    Implementation of the identity block as defined in Figure 3
    
    Arguments:
    X -- input tensor of shape (m, n_H_prev, n_W_prev, n_C_prev)
    f -- integer, specifying the shape of the middle CONV's window for the main path
    filters -- python list of integers, defining the number of filters in the CONV layers of the main path
    stage -- integer, used to name the layers, depending on their position in the network
    block -- string/character, used to name the layers, depending on their position in the network
    
    Returns:
    X -- output of the identity block, tensor of shape (n_H, n_W, n_C)
    """
    
    # defining name basis
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    
    # Retrieve Filters
    F1, F2, F3 = filters
    
    # Save the input value. You'll need this later to add back to the main path. 
    X_shortcut = X
    
    # First component of main path
    X = Conv2D(filters = F1, kernel_size = (1, 1), strides = (1,1), padding = 'valid', name = conv_name_base + '2a', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2a')(X)
    X = Activation('relu')(X)
    
    ### START CODE HERE ###
    
    # Second component of main path (≈3 lines)
    X = Conv2D(filters= F2,kernel_size =(f,f), strides=(1,1),padding='same',name= conv_name_base + '2b',kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis= 3, name = bn_name_base + '2b')(X)
    X = Activation('relu')(X)

    # Third component of main path (≈2 lines)
    X = Conv2D(filters=F3,kernel_size=(1,1),strides=(1,1),padding='valid',name = conv_name_base  + '2c', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3,name=bn_name_base + '2c')(X)

    # Final step: Add shortcut value to main path, and pass it through a RELU activation (≈2 lines)
    X = Add()([X,X_shortcut])
    X = Activation('relu')(X)
    
    ### END CODE HERE ###
    
    return X

In [185]:
# GRADED FUNCTION: convolutional_block

def convolutional_block(X, f, filters, stage, block, s = 2):
    """
    Implementation of the convolutional block as defined in Figure 4
    
    Arguments:
    X -- input tensor of shape (m, n_H_prev, n_W_prev, n_C_prev)
    f -- integer, specifying the shape of the middle CONV's window for the main path
    filters -- python list of integers, defining the number of filters in the CONV layers of the main path
    stage -- integer, used to name the layers, depending on their position in the network
    block -- string/character, used to name the layers, depending on their position in the network
    s -- Integer, specifying the stride to be used
    
    Returns:
    X -- output of the convolutional block, tensor of shape (n_H, n_W, n_C)
    """
    
    # defining name basis
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    
    # Retrieve Filters
    F1, F2, F3 = filters
    
    # Save the input value
    X_shortcut = X


    ##### MAIN PATH #####
    # First component of main path 
    X = Conv2D(F1, (1, 1), strides = (s,s), name = conv_name_base + '2a', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2a')(X)
    X = Activation('relu')(X)
    
    ### START CODE HERE ###

    # Second component of main path (≈3 lines)
    X = Conv2D(F2,(f,f) , strides=(1,1),padding='same', name= conv_name_base + '2b', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3,name= bn_name_base + '2b')(X)
    X = Activation('relu')(X)

    # Third component of main path (≈2 lines)
    X = Conv2D(F3,(1,1),strides=(1,1), padding='valid',name= conv_name_base + '2c',kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3 , name = bn_name_base + '2c')(X)

    ##### SHORTCUT PATH #### (≈2 lines)
    X_shortcut = Conv2D(F3,(1,1),strides=(s,s) ,padding='valid',name = conv_name_base + '1',kernel_initializer=glorot_uniform(seed=0))(X_shortcut)
    X_shortcut = BatchNormalization(axis = 3, name = bn_name_base + '1')(X_shortcut)

    # Final step: Add shortcut value to main path, and pass it through a RELU activation (≈2 lines)
    X = Add()([X,X_shortcut])
    X = Activation('relu')(X)
    
    ### END CODE HERE ###
    
    return X

In [186]:
# GRADED FUNCTION: ResNet50

def ResNet50(input_shape = (225, 300, 3), classes = 7):
    """
    Implementation of the popular ResNet50 the following architecture:
    CONV2D -> BATCHNORM -> RELU -> MAXPOOL -> CONVBLOCK -> IDBLOCK*2 -> CONVBLOCK -> IDBLOCK*3
    -> CONVBLOCK -> IDBLOCK*5 -> CONVBLOCK -> IDBLOCK*2 -> AVGPOOL -> TOPLAYER

    Arguments:
    input_shape -- shape of the images of the dataset
    classes -- integer, number of classes

    Returns:
    model -- a Model() instance in Keras
    """
    
    # Define the input as a tensor with shape input_shape
    X_input = Input(input_shape)

    
    # Zero-Padding
    X = ZeroPadding2D((3, 3))(X_input)
    
    # Stage 1
    X = Conv2D(64, (7, 7), strides = (2, 2), name = 'conv1', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = 'bn_conv1')(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((3, 3), strides=(2, 2))(X)

    # Stage 2
    X = convolutional_block(X, f = 3, filters = [64, 64, 256], stage = 2, block='a', s = 1)
    X = identity_block(X, 3, [64, 64, 256], stage=2, block='b')
    X = identity_block(X, 3, [64, 64, 256], stage=2, block='c')

    ### START CODE HERE ###

    # Stage 3 (≈4 lines)
    X = convolutional_block(X, f=3, filters=[128,128,512],stage=3,s=2,block='a')
    X = identity_block(X,3,[128,128,512], stage=3,block='b')
    X = identity_block(X,3,[128,128,512], stage=3,block='c')
    X = identity_block(X,3,[128,128,512], stage=3,block='d')

    # Stage 4 (≈6 lines)
    X = convolutional_block(X, f=3, filters=[256, 256, 1024],stage=4,s=2,block='a')
    X = identity_block(X, 3,[256, 256, 1024],stage=4,block='b')
    X = identity_block(X, 3,[256, 256, 1024],stage=4,block='c')
    X = identity_block(X, 3,[256, 256, 1024],stage=4,block='d')
    X = identity_block(X, 3,[256, 256, 1024],stage=4,block='e')
    X = identity_block(X, 3,[256, 256, 1024],stage=4,block='f')

    # Stage 5 (≈3 lines)
    X = convolutional_block(X, f=3, filters=[512, 512, 2048],stage=5,s=2,block='a')
    X = identity_block(X, 3,[512, 512, 2048],stage=5,block='b')
    X = identity_block(X, 3,[512, 512, 2048],stage=5,block='c')

    # AVGPOOL (≈1 line). Use "X = AveragePooling2D(...)(X)"
    X = AveragePooling2D(pool_size=(2, 2),name='avg_pool')(X)
    
    ### END CODE HERE ###

    # output layer
    X = Flatten()(X)
    X = Dense(classes, activation='softmax', name='fc' + str(classes), kernel_initializer = glorot_uniform(seed=0))(X)
    
    
    # Create model
    model = Model(inputs = X_input, outputs = X, name='ResNet50')

    return model

In [187]:
model = ResNet50(input_shape = (225, 300, 3), classes = 7)

In [188]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [189]:
steps_per_epoch=tf.ceil(image_count/BATCH_SIZE).numpy()
steps_per_epoch = int(steps_per_epoch)
steps_per_epoch

313

In [190]:
import os

In [191]:

checkpoint_dir = './training_checkpoints'

checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [192]:
#model.fit(ds,epochs = 3, steps_per_epoch = steps_per_epoch, batch_size = BATCH_SIZE, callbacks=[checkpoint_callback])

In [194]:
from oauth2.client import GoogleCredentials
from googleapiclient import discovery
from googleapiclient import errors

ImportError: No module named 'oauth2'

In [196]:
import oauth2

ImportError: No module named 'oauth2'